<a href="https://colab.research.google.com/github/SaraGhR/hip-fracture-care-episodes/blob/main/hip_fracture_analyzing_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as sm
from sklearn.model_selection import train_test_split

In [ ]:
!pip install pyreadstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.4/666.4 kB 12.1 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('star.csv')
df.shape

In [ ]:
for i in range(0,515):
  patient = df.iloc[i,1:]
  plt.plot(patient.values)
  plt.title("Patient Care Episode Trajectory")
  plt.xlabel("Days after fracture")
  plt.ylabel("Status (0=home,...,4=dead)")
  plt.show()

In [ ]:
sns.heatmap(df.iloc[:,1:515], cmap="coolwarm", cbar=True)
plt.title("all days for all patients")
plt.show()

#Clustering

stable situation, only gets better, only gets worse, unstable situation